# Chapter 5. Support Vector Machines

Una máquina de soporte vectorial (SVM) es un modelo de aprendizaje automático potente y versátil, capaz de realizar clasificación lineal o no lineal, regresión e incluso detección de valores atípicos.

Es uno de los modelos más populares en Machine Learning, y cualquier persona interesada en Machine Learning debería tenerlo en su caja de herramientas.

Las SVM son particularmente adecuadas para la clasificación de conjuntos de datos complejos de tamaño pequeño o mediano.

Este capítulo explicará los conceptos básicos de las SVM, cómo usarlas y cómo funcionan.

## Linear SVM Classification

The fundamental idea behind SVMs is best explained with some pictures. 

Figure 5-1 shows part of the iris dataset that was introduced at the end of Chapter 4. 

The two classes can clearly be separated easily with a straight line (they are linearly separable). The left plot shows the decision boundaries of three possible linear classifiers. 

The model whose decision boundary is represented by the dashed line is so bad that it does not even separate the classes properly. 

The other two models work perfectly on this training set, but their decision boundaries come so close to the instances that these models will probably not perform as well on new instances. 

In contrast, the solid line in the plot on the right represents the decision boundary of an SVM classifier; this line not only separates the two classes but also stays as far away from the closest training instances as possible. 

You can think of an SVM classifier as fitting the widest possible street (represented by the parallel
dashed lines) between the classes. This is called large margin classification.

<img src = 'https://github.com/marco-canas/didactica_ciencia_datos/blob/main/referentes/geron/part_1/chap_5_sv/figure_5_1.jpg?raw=true'>

Notice that adding more training instances “off the street” will not affect the decision boundary at all: it is fully determined (or “supported”) by the instances located on the edge of the street. 

These instances are called the support vectors (they are circled in Figure 5-1).

<img src = 'https://github.com/marco-canas/didactica_ciencia_datos/blob/main/referentes/geron/part_1/chap_5_sv/figure_5_2.jpg?raw=true'>

## WARNING

SVMs are sensitive to the feature scales, as you can see in Figure 5-2: in the left plot, the vertical scale is much larger than the horizontal scale, so the widest possible street is close to horizontal. 

After feature scaling (e.g., using Scikit-Learn’s StandardScaler), the decision boundary in the right plot looks much better.

## Soft Margin Classification

If we strictly impose that all instances must be off the street and on the right side, this is
called hard margin classification. 

There are two main issues with hard margin classification. 

First, it only works if the data is linearly separable. Second, it is sensitive to outliers. 

Figure 5-3 shows the iris dataset with just one additional outlier: on the left, it is impossible to find a hard margin; on the right, the decision boundary ends up very different from the one we saw in Figure 5-1 without the outlier, and it will probably not generalize as well.

<img src = ''>

To avoid these issues, use a more flexible model. 

The objective is to find a good balance between keeping the street as large as possible and limiting the margin violations (i.e., instances that end up in the middle of the street or even on the wrong side). 

This is called soft margin classification.

When creating an SVM model using Scikit-Learn, we can specify a number of
hyperparameters. 

C is one of those hyperparameters. 

If we set it to a low value, then we end up with the model on the left of Figure 5-4. 

With a high value, we get the model on the right. 

Margin violations are bad. It’s usually better to have few of them. 

However, in this case the model on the left has a lot of margin violations but will probably generalize better.

<img src = ''>

### TIP

If your SVM model is overfitting, you can try regularizing it by reducing C.

The following Scikit-Learn code loads the iris dataset, scales the features, and then trains a linear SVM model (using the LinearSVC class with C=1 and the hinge loss function, described shortly) to detect Iris virginica flowers:


In [1]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
iris = datasets.load_iris()
X = iris["data"][:, (2, 3)] # petal length, petal width
y = (iris["target"] == 2).astype(np.float64) # Iris virginica
svm_clf = Pipeline([
("scaler", StandardScaler()),
("linear_svc", LinearSVC(C=1, loss="hinge")),
])
svm_clf.fit(X, y)


Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge'))])